# 160 : Demo Nested-NER Multihead

In [ ]:
#!pip install ipywidgets

In [1]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [7]:
models = ['nlpso/m1_ind_layers_ref_cmbert_io',
 'nlpso/m1_ind_layers_ref_cmbert_iob2',
 'nlpso/m1_ind_layers_ref_ptrn_cmbert_io',
 'nlpso/m1_ind_layers_ref_ptrn_cmbert_iob2',
 'nlpso/m1_ind_layers_ocr_cmbert_io',
 'nlpso/m1_ind_layers_ocr_cmbert_iob2',
 'nlpso/m1_ind_layers_ocr_ptrn_cmbert_io',
 'nlpso/m1_ind_layers_ocr_ptrn_cmbert_iob2',
]

Choose an model on the HuggingFace among the following list :

In [9]:
import ipywidgets as widgets

model = widgets.RadioButtons(
            options=models,
            layout={'width': 'max-content'}
        )
model

RadioButtons(layout=Layout(width='max-content'), options=('nlpso/m1_ind_layers_ref_cmbert_io', 'nlpso/m1_ind_l…

In [14]:
FINETUNED_MODEL_N1 = model.value + '_level_1'
FINETUNED_MODEL_N2 = model.value + '_level_2'
print(FINETUNED_MODEL_N1)
print(FINETUNED_MODEL_N2)

nlpso/m1_ind_layers_ref_cmbert_io_level_1
nlpso/m1_ind_layers_ref_cmbert_io_level_2


### Data

In [15]:
samples = """☞  T Dufant (Victor), libraire, r. du Gros-Che- net. 2. JO \n
Dutay, essayeur du commerce, place Dau-  - píině, 5. DOB \n
Dulay, chandronnier, r. du Pont- aux- Chuux, 15. SI \n
Dufay (V.e), grenetière, r. du Fauh.-S. Denis, 20. 372 \n
Y ☞ Dnten,charentier, 1. Montmartre, 89. 318 \n
Dufey fils, bijoutier, passage de la Réunion A \n
Dnley, boucher, r. S.&quot;Louis-au-Marais,3. * \n
☞ Duffand, bijonteer, r. S.-Martin, 107. 284 \n
Duftaud, maçon, r. S-Honore  353. O \n
Dullaut, chandronnier, r. de la Sourdière I- ( ☞T4 \n
lukils. limonauier, 1. des Colonnes, 6. J9 \n
— T Dullocu, architecte, r. Basse-Porte-S.De- — Ris, 28. \n
Duflot, loueur de carrosses, r. de Paradis- 505 Poissonnière, 22. \n
Dufort, bottier, Palais-R., gal. vitrée, 215. A \n
Dufort fils, bottier (exp. 1819.), r. J.-J.. Rousscan. 18. 290 \n
* Dufort, tabletier, r. Jean-Pain-Mollet, 10. 4O \n
Dufossé, teiniurier, r. du Faub.-Montmar- tre , 63. 483 \n
Duſour el Besnard, march, de bois à bruler, quai de la Tournelle, 17. etr. des Fossés- SBernard. 11. Dí \n
Dufour jeune, march, de bois à brüler de la Pépinière, 53. 7 r. \n
Dufour (Mad.), march. de chapeaux de paitle, passage dn Panorama, 11. IO \n
Dutour, thapelier, r. S.-Honore 383. 3og \n
Dufour, cirier, r. de PArbre-Sec, 6o. 316 \n
Dufour et co., commiss., r. des Contures-  DAO S. Dervajs, 18. \n
Dulour, corropeir, 1. des Vertusy1d 3 \n
Dufour (Charles), épicier, r. S.-Denis, Jo7. ☞ 332 \n
Dufour, fab. d&apos;éventails (erp. 1819) Beaubourg, 48.  360 \n
Dufour, faiencter, r. S.-Anne , 50. 302 \n
Dufour, A. Sarnt,r. Ventadour, ☞7 \n
Dufour (Gabriel), libraire, r. de Vaugirard, A \n
Dufour, maçon, r. S.-Jean-Baptiste, 4 402 \n
Dufour, architecte, r. de Chartres-S.-Ho- 2781 40a nore, 12. (Elig.) \n
Dufour (Mad.), nouveautes, 1. Neuve-des- H12 Petits-Champs, 4. \n
Dufour Chabrol, papetier, r. S.-Martin ☞ 4 \n
Dufour (J.) et co,, fab. de papiers peints, ((1819); r. Beauvean-S. Antoine, 19. \n
\n
Waring et Gillow Limited, ameublements an- glais, éctairage électrique et appareils sa- nitaires, r. Glück, 8 et boul. Haussmann, 31, TELÉPH] 244.45; succursale, r. Bleue, 7. WARISON�PARIS. TĚLÉPH 244.40. Adr.\n
Barruel (Paul), fabr. de produis chimiques et de siccatif zumatique, adjoint au maire du 15e arrond., Alleray, 70; dépót chez H. Suillot sle-Croix-de-la-Bretonnerič. 21 \n
Marchand (Mme) née Louise Scelle, en gros et détail, r. 77, et à Bercy, r. de Bourgogne, 25. d'Enfer, \n
Very (A.). ingenieur-constructeur, r. d'Angou- lême, 66, cité d'Angouléme, 6. TĚLÉPH 420.82. \n
Vacherie de Royaumont, Brémontler. 16; Chaussée-d'Antin, 68; Phals- bourg, 3 ; blv. Poissonnière, 1, et blv. St-Martin, 15. \n
Budan, inspect.-gén. des études, r. Charlot. 18 (au Marais). \n
"""

### Tools

In [18]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TokenClassificationPipeline
from xmlize_util import get_NER_tags, xmlize_multilevel

LIMIT = 100

#Tools n1
tokenizer1 = AutoTokenizer.from_pretrained(FINETUNED_MODEL_N1)
model1 = AutoModelForTokenClassification.from_pretrained(FINETUNED_MODEL_N1)
apply1 = TokenClassificationPipeline(model=model1, tokenizer=tokenizer1, aggregation_strategy='simple', ignore_labels=[''])

#Tools n2
tokenizer2 = AutoTokenizer.from_pretrained(FINETUNED_MODEL_N2)
model2 = AutoModelForTokenClassification.from_pretrained(FINETUNED_MODEL_N2)
apply2 = TokenClassificationPipeline(model=model2, tokenizer=tokenizer2, aggregation_strategy='simple', ignore_labels=[''])

for s in samples.split("\n"):
    print(s)
    res = []
    res.append(apply1(s))
    res.append(apply2(s))
    levels, num_levels = get_NER_tags(s,res)

    if len(levels['niv_1']) > 0:
        print(xmlize_multilevel(levels,num_levels))
    print('')

Downloading:   0%|          | 0.00/710 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/811k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/241 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/241 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

☞  T Dufant (Victor), libraire, r. du Gros-Che- net. 2. JO 
<TITREH>☞</TITREH><PER> T Dufant (Victor</PER>), <ACT>libraire</ACT>, <SPAT><LOC>r. du Gros-Che- net</LOC>. <CARDINAL>2</CARDINAL>.<LOC> JO</LOC></SPAT> 



Dutay, essayeur du commerce, place Dau-  - píině, 5. DOB 
<PER>Dutay</PER>, <ACT>essayeur du commerce</ACT>, <SPAT><LOC>place Dau-  - píině</LOC>, <CARDINAL>5</CARDINAL>. DOB</SPAT> 



Dulay, chandronnier, r. du Pont- aux- Chuux, 15. SI 
<PER>Dulay</PER>, <ACT>chandronnier</ACT>, <SPAT><LOC>r. du Pont- aux- Chuux</LOC>, <CARDINAL>15</CARDINAL>. SI</SPAT> 



Dufay (V.e), grenetière, r. du Fauh.-S. Denis, 20. 372 
<PER>Dufay (V.e),</PER> <ACT>grenetière</ACT>, <SPAT><LOC>r. du Fauh.-S. Denis</LOC>, <CARDINAL>20</CARDINAL></SPAT>.<CARDINAL> 372</CARDINAL> 



Y ☞ Dnten,charentier, 1. Montmartre, 89. 318 
<PER>Y<TITREH> </TITREH>☞ Dnten</PER>,<ACT>charentier</ACT>, <SPAT><LOC>1. Montmartre</LOC>, <CARDINAL>89</CARDINAL></SPAT>. <CARDINAL>318</CARDINAL> 



Dufey fils, bijout

<PER>Vacherie de Royaumont</PER>, <SPAT><LOC>Brémontler</LOC>.<CARDINAL> 16</CARDINAL></SPAT>; <SPAT>Chaussée-d'Antin, <CARDINAL>68</CARDINAL></SPAT>; <SPAT><LOC>Phals- bourg</LOC>, <CARDINAL>3</CARDINAL></SPAT> ; <SPAT><LOC>blv. Poissonnière</LOC>, <CARDINAL>1</CARDINAL></SPAT>, et <SPAT><LOC>blv. St-Martin</LOC>, <CARDINAL>15</CARDINAL></SPAT>. 



Budan, inspect.-gén. des études, r. Charlot. 18 (au Marais). 
<PER>Budan</PER>, <ACT>inspect.-gén. des études</ACT>, <SPAT><LOC>r. Charlot.</LOC> <CARDINAL>18</CARDINAL> <LOC>(au Marais</LOC></SPAT>). 





